# Tool to obtain valid launch times of a satellite A to target specific orbit plane to trail another satellite B in RAAN

Code flow:
1. Pull TLE data from Celestrak
2. Propagate target satellite B's RAAN for specified time with J2 effect
3. Obtain RAAN target plane for satellite A to launch into 
4. Match RAAN to LAN of launch site within tolerance, correcting LAN with the angle between vernal equinox and prime meridian
5. Obtain time stamps when condition in step 4 are valid. These are valid launch times subject to the tolerance chosen. 


Required inputs:
data_propagate_J2_list - list of strings, in form "YYYYMMDD"
cat_id - int, NORAD ID of satellite. NEUSAR--> 52937

In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [2]:
from propagate_J2 import propagate_J2

data_propagate_J2_list = ["20221101", "20221201", \
"20230101", "20230201", "20230301", "20230401", "20230501"]
cat_id = 52937 



# Propagate orbit plane's RAAN with J2 effect
Using the most current TLE from Celestrak, we output the RAAN due to J2 on a date in the future. 

The rate of change of RAAN with time: 
\begin{equation}
\frac{d\omega}{dt} = -\frac{3}{2}J_2(\frac{R_E^2}{a^2(1-e^2)})^2\times \tilde{n}\cos{i}
\end{equation}

$R_E$ denotes the radius of Earth (km)  
$\mu_E$ denotes the gravitational constant of the earth  
$\tilde{n}$ denotes the perturbed mean motion. Some authors use mean motion rather than its perturbed version for simplicity, however this introduces errors over longer periods of time. Using the perturbed mean motion also gives a closer value to HPOP, which serves as a ground truth. (K.F. Wakker, Page 631, Fundamentals of Astrodynamics, TU Delft 2015)

The perturbed mean motion is related to the mean motion $n$, via: 
\begin{equation}
\tilde{n} = n [1 + \frac{3}{4}J_2(\frac{R}{a})^2(1-e^2)^(-3/2)(3\cos(i)^2-1)]
\end{equation}

# Obtaining target orbit plane

We are interested in launching our satellite into a target plane, displaced by 120 $\degree$ from NeuSAR. This is done by simply subtracting 120 $\degree$ from NeuSAR's RAAN.

In [3]:
overall_df = propagate_J2(data_propagate_J2_list, cat_id)

RAAN of cat id 52937 is 250.84685 deg on 01/11/2022, 08:39 UTC, 26 days from today.
RAAN of cat id 52937 is 32.36360 deg on 01/12/2022, 08:39 UTC, 56 days from today.
RAAN of cat id 52937 is 166.59757 deg on 01/01/2023, 08:39 UTC, 87 days from today.
RAAN of cat id 52937 is 300.83154 deg on 01/02/2023, 08:39 UTC, 118 days from today.
RAAN of cat id 52937 is 96.91384 deg on 01/03/2023, 08:39 UTC, 146 days from today.
RAAN of cat id 52937 is 231.14781 deg on 01/04/2023, 08:39 UTC, 177 days from today.
RAAN of cat id 52937 is 12.66456 deg on 01/05/2023, 08:39 UTC, 207 days from today.


# Valid launch times
Final goal is to get valid times to launch. Since the launch site is fixed, and we know the target orbit plane we require and how it evolves over time, we are able to then match the target orbit plane's RAAN with the Longitude of ascending node (LAN) from launching satellite into orbit at a certain time. The time of day and date affects the LAN, so the launch window is dependent on both the date and time of day.  

With a variable del_RAAN to set the tolerance for the orbit plane, which accounts for uncertainties such as the launch trajectory, we can select valid launch times where the target RAAN matches the LAN within tolerance. This then becomes the valid launch times.  

It is recommended for del_RAAN $ > 4$ as the launch duration is slated to be 20mins +, which translates to a rotation of about 2.5 $\degree$ in longitude from the time of launch to end.  


In [ ]:
from propagate_J2 import output_launch_times

RAAN_tol = 5 #degrees
launch_site_coords = (13.73204, 80.23621)
launch_azimuth = 102 #degree
launch_data = output_launch_times(overall_df, launch_site_coords, RAAN_tol)
valid_launch_times = launch_data["T_prop_T2"]

# Verification of analytical method with HPOP
